In [2]:
import tensorflow_data_validation as tfdv
import pandas as pd
import datetime

from tensorflow_data_validation.utils import slicing_util
from typing import List, Optional, Text, Union, Dict, Iterable, Mapping

from tensorflow_metadata.proto.v0 import schema_pb2

In [3]:
tfdv.__version__

'0.22.0'

## Configure environment settings

In [4]:
base_schema_path = 'gs://mlops-dev-workspace/drift-monitor/templates/schema/schema.pbtxt'
baseline_stats_path = 'gs://mlops-dev-workspace/drift-monitor/templates/baseline_stats/stats.pbtxt'
schema_path = 'gs://mlops-dev-workspace/drift-monitor/schema/schema.pbtxt'

In [5]:
schema = tfdv.load_schema_text(base_schema_path)

### Fine tune the schema

In [6]:
# Do your stuff
for feature in schema.feature:
    print(feature.name)

Elevation
Aspect
Slope
Horizontal_Distance_To_Hydrology
Vertical_Distance_To_Hydrology
Horizontal_Distance_To_Roadways
Hillshade_9am
Hillshade_Noon
Hillshade_3pm
Horizontal_Distance_To_Fire_Points
Soil_Type
Wilderness_Area


In [7]:
#new_feature = schema.feature.add()
#new_feature.name = 'time_slice'
#new_feature.type = schema_pb2.FeatureType.BYTES

In [8]:
path = tfdv.FeaturePath("Wilderness_Area")
#

### Save the updated schema

In [9]:
tfdv.write_schema_text(schema, schema_path)

## Run the job

In [10]:
project = 'mlops-dev-env'
log_table = 'data_validation.test1'
model = 'covertype_tf'
version = 'v3'
start_time = '2020-05-25T16:01:10'
end_time = '2020-05-25T22:50:30'
output_path = 'gs://mlops-dev-workspace/drift-monitor/output/test'
baseline_stats_file = 'gs://mlops-dev-workspace/drift-monitor/baseline_stats/stats.pbtxt'
time_window = '60m'

In [11]:
!gsutil ls {schema_file}

CommandException: "ls" command does not support "file://" URLs. Did you mean to use a gs:// URL?


In [12]:
!python ../run.py \
--project={project} \
--request_response_log_table={log_table} \
--model={model} \
--version={version}\
--start_time={start_time} \
--end_time={end_time} \
--output_path={output_path} \
--schema_file={schema_path} \
--baseline_stats_file={baseline_stats_file} \
--time_window={time_window}

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:root:File gs://mlops-dev-workspace/drift-monitor/baseline_stats/stats.pbtxt did not look like a TFRecord. Try reading as a plain file.
INFO:root:Starting the request-response log analysis pipeline...
INFO:root:Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:apache_beam.internal.gcp.auth:Setting socket default timeout to 60 seconds.
INFO:apache_beam.internal.gcp.auth:socket default timeout is 60.0 seconds.
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'data_validation'
 projectId: 'mlops-dev-env'
 tableId: 'test1'> referenced by query 
        SELECT FORMAT_TIMESTAMP("%G-%m-%d %T", time) as time, raw_data
        FROM 
            `data_vali

## Analyze results

In [19]:
stats = tfdv.load_statistics(stats_path)
all_examples_stats = tfdv.get_slice_stats(stats, "All Examples")
tfdv.visualize_statistics(all_examples_stats)

In [20]:
anomalies_path = output_path + '/' + 'anomalies.pbtxt'
!gsutil ls {anomalies_path}

gs://mlops-dev-workspace/drift-monitor/output/test/anomalies.pbtxt


In [16]:
stats_path = output_path + '/' + 'stats.pb'
!gsutil ls {stats_path}

gs://mlops-dev-workspace/drift-monitor/output/test/stats.pb


In [21]:
anomalies = tfdv.load_anomalies_text(anomalies_path)
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'Wilderness_Area',Unexpected string values,Examples contain values missing from the schema: Others (~1%).
'Soil_Type',Unexpected string values,"Examples contain values missing from the schema: 3501 (<1%), 3502 (<1%), 7710 (<1%), 8708 (<1%)."
